# Multiary complex model of GPCR signaling activations
## Supplemental Material for Watabe et al. Multiary complex formations in GPCR signaling activations. arXiv:xxx.xxxxxx (2020).

This notebook provides the source codes of a multiary complex model of GPCR signaling activations via dimer formation of ligand-induced receptors, represented by a multivalent form of physical observables under basis vectors of various aggregated receptor states.

### The simplest ternary complex model

In [1]:
#!/usr/bin/env python
# coding: utf-8

# Multiary Complex Model in GPCR signaling activation
#get_ipython().run_line_magic('matplotlib', 'inline')
import os
import sys
import numpy

from parameters import ParConfigs

from ecell4 import *
from ecell4_base.core import *
from ecell4_base import gillespie, ode

In [2]:
# this is the ternary complex model (TCM): N=0
N = 0.00

# Inputs :
# Ligand and G-protein concentrations
L = 1.00
G = 1.00

# scaling factor
alpha = 1.00

# The lumped dimensionless parameter
kx = 0.00

# get model-parameters
par = ParConfigs()

mols = ["r", "R", "Gr", "GR"]

obs0 = numpy.zeros(shape=(len(mols)))
obs0[0] = par.get_r0()

###############################################################
print ('Model-order: N =', N)
print ('Ligand: L =', L)
print ('G-protein: G =', G)
print ('alpha-factor: a =', alpha)
print ('Initial condition:')
for i in range(len(mols)) :
    print ('  ', mols[i], ': ', obs0[i])
###############################################################

# ligands-receptors first-order coupling
dl0, dl1, dl2 = par.get_dl()
kl0, kl1, kl2 = par.get_kl(100, 100)

# Gproteins-receptors first-order coupling
dg0, dg1 = par.get_dg()
kg0, kg1 = par.get_kg(100)

# ligand-(Gproteins-receptors) first-order coupling
da0, da1, da2 = par.get_da()
ka0, ka1, ka2 = par.get_ka(alpha, 100, 100)

# Gproteins-(ligand-receptors) first-order coupling
db0, db1, db2 = par.get_db()
kb0, kb1, kb2 = par.get_kb(alpha, 100, 100)

## receptor-receptor second-order coupling
#dx0, dx1, dx2 = par.get_dx()
#kx0, kx1, kx2 = par.get_kx(kx, N)
#
## receptor-Gprotein second-order coupling
#dy0, dy1, dy2, dy3 = par.get_dy()
#ky0, ky1, ky2, ky3 = par.get_ky(1, 1, 1, 1)

Model-order: N = 0.0
Ligand: L = 1.0
G-protein: G = 1.0
alpha-factor: a = 1.0
Initial condition:
   r :  49770.0
   R :  0.0
   Gr :  0.0
   GR :  0.0


In [3]:
with reaction_rules():
    # TCM (Ternary complex model)
    r  == R  | (kl0*L, dl0)
    R  == GR | (ka0*G, da0)
    r  == Gr | (kg0*G, dg0)
    Gr == GR | (kb0*L, db0)

m = get_model()
show(m)


r > R | 1.0
R > r | 1.0
R > GR | 1.0
GR > R | 1.0
r > Gr | 1.0
Gr > r | 1.0
Gr > GR | 1.0
GR > Gr | 1.0


In [4]:
# Create simulator (ODE)
w = ode.World()
w.bind_to(m)
sim = ode.Simulator(w)

In [5]:
# Initialization
print ("Initial condition")
for i in range(len(mols)) :
  obs0_i = int(round(obs0[i]))
  w.add_molecules(Species("%s" % (mols[i])), obs0_i)
  print (mols[i], ':', obs0_i)

sim.initialize()

Initial condition
r : 49770
R : 0
Gr : 0
GR : 0


In [6]:
# Run simulator
period = 50. # sec

obs = FixedIntervalNumberObserver(1.0, mols)
sim.run(period, [obs])
obs_data = numpy.array(obs.data())

time = obs_data[:,0]
R0 = obs_data[:,1]
RM = numpy.zeros(shape=(len(time)))
RD = obs_data[:,2]
G0 = obs_data[:,3]
GM = obs_data[:,4]
GD = numpy.zeros(shape=(len(time)))

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
from numpy import array

data = array(obs.data()).T

plt.plot(data[0], RM+RD+GM+GD, "b-", label="Bound states")
plt.plot(data[0], RM, "-", label="M")
plt.plot(data[0], RD, "-", label="D")
plt.plot(data[0], GM, "-", label="G•M")
plt.plot(data[0], GM, "-", label="G•D")

plt.xlabel("Time [sec]")
plt.ylabel("Bounds")
plt.xlim(data[0][0], data[0][-1])
plt.legend(loc="best", shadow=True)
plt.show()

### The multiary complex model

#### Monovalent model (N = 1 and kx > 0)

In [ ]:
# this is the monovalent model : N=1
N = 1.00

# Inputs :
# Ligand and G-protein concentrations
L = 1.00
G = 1.00

# scaling factor
alpha = 1.00

# The lumped dimensionless parameter (kx > 0)
kx = 0.001

# get model-parameters
par = ParConfigs()

mols = ["r", "rr", "R", "rR", "RR", "Gr", "Grr", "GR", "GrR", "GRR"]

obs0 = numpy.zeros(shape=(len(mols)))
obs0[0] = par.get_r0()

###############################################################
print ('Model-order: N =', N)
print ('Ligand: L =', L)
print ('G-protein: G =', G)
print ('alpha-factor: a =', alpha)
print ('Initial condition:')
for i in range(len(mols)) :
    print ('  ', mols[i], ': ', obs0[i])
###############################################################

# ligands-receptors first-order coupling
dl0, dl1, dl2 = par.get_dl()
kl0, kl1, kl2 = par.get_kl(100, 100)

# Gproteins-receptors first-order coupling
dg0, dg1 = par.get_dg()
kg0, kg1 = par.get_kg(100)

# ligand-(Gproteins-receptors) first-order coupling
da0, da1, da2 = par.get_da()
ka0, ka1, ka2 = par.get_ka(alpha, 100, 100)

# Gproteins-(ligand-receptors) first-order coupling
db0, db1, db2 = par.get_db()
kb0, kb1, kb2 = par.get_kb(alpha, 100, 100)

# receptor-receptor second-order coupling
dx0, dx1, dx2 = par.get_dx()
kx0, kx1, kx2 = par.get_kx(kx, N)

# receptor-Gprotein second-order coupling
dy0, dy1, dy2, dy3 = par.get_dy()
ky0, ky1, ky2, ky3 = par.get_ky(1, 1, 1, 1)

In [ ]:
with reaction_rules():

    # TCM (Ternary complex model)
    r  == R  | (kl0*L, dl0)
    R  == GR | (ka0*G, da0)
    r  == Gr | (kg0*G, dg0)
    Gr == GR | (kb0*L, db0)

    # receptor dimerization
    r + r == rr | (kx0, dx0)
    R + r == rR | (kx1, dx1)
    R + R == RR | (kx2, dx2)

    rr == rR | (kl1*L, dl1)
    rR == RR | (kl2*L, dl2)

    # dimers coupled with G-proteins
    rr == Grr | (kg1*G, dg1)
    rR == GrR | (kb1*G, db1)
    RR == GRR | (kb2*G, db2)

    Gr + r == Grr | (ky0, dy0)
    GR + r == GrR | (ky1, dy1)
    Gr + R == GrR | (ky2, dy2)
    GR + R == GrR | (ky3, dy3)

    Grr == GrR | (ka1*L, da1)
    GrR == GRR | (ka2*L, da2)

m = get_model()
show(m)

#### Bivalent model (N = 2 and kx > 0)

In [ ]:
# this is the bivalent model : N=2
N = 2.00

mols = ["r", "rr", "rrr", "rrrr", \
        "R", "rR", "rrR", "rrrR", \
        "RR", "rRR", "rrRR", \
        "Gr", "Grr", "Grrr", "Grrrr", \
        "GR", "GrR", "GrrR", "GrrrR", \
        "GRR", "GrRR", "GrrRR"]

In [ ]:
with reaction_rules():

    # TCM (Ternary complex model)
    r  == R  | (kl0*L, dl0)
    R  == GR | (ka0*G, da0)
    r  == Gr | (kg0*G, dg0)
    Gr == GR | (kb0*L, db0)

    # receptor dimerization
    r  + r  == rr   | (kx0, dx0)
    r  + rr == rrr  | (kx0, dx0)
    rr + r  == rrr  | (kx0, dx0)
    rr + rr == rrrr | (kx0, dx0)

    R  + r  == rR   | (kx1, dx1)
    R  + rr == rrR  | (kx1, dx1)
    rR + r  == rrR  | (kx1, dx1)
    rR + rr == rrrR | (kx1, dx1)

    R  + R  == RR   | (kx2, dx2)
    R  + rR == rRR  | (kx2, dx2)
    rR + R  == rRR  | (kx2, dx2)
    rR + rR == rrRR | (kx2, dx2)

    rr   == rR   | (kl1*L, dl1)
    rrr  == rrR  | (kl1*L, dl1)
    rrrr == rrrR | (kl1*L, dl1)

    rR   == RR   | (kl2*L, dl2)
    rrR  == rRR  | (kl2*L, dl2)
    rrrR == rrRR | (kl2*L, dl2)

    # dimers coupled with G-proteins
    rr   == Grr   | (kg1*G, dg1)
    rrr  == Grrr  | (kg1*G, dg1)
    rrrr == Grrrr | (kg1*G, dg1)

    rR   == GrR   | (kb1*G, db1)
    rrR  == GrrR  | (kb1*G, db1)
    rrrR == GrrrR | (kb1*G, db1)

    RR   == GRR   | (kb2*G, db2)
    rRR  == GrRR  | (kb2*G, db2)
    rrRR == GrrRR | (kb2*G, db2)

    Gr  + r  == Grr   | (ky0, dy0)
    Gr  + rr == Grrr  | (ky0, dy0)
    Grr + r  == Grrr  | (ky0, dy0)
    Grr + rr == Grrrr | (ky0, dy0)

    GR  + r  == GrR   | (ky1, dy1)
    GR  + rr == GrrR  | (ky1, dy1)
    GrR + r  == GrrR  | (ky1, dy1)
    GrR + rr == GrrrR | (ky1, dy1)

    R  + Gr  == GrR   | (ky2, dy2)
    R  + Grr == GrrR  | (ky2, dy2)
    rR + Gr  == GrrR  | (ky2, dy2)
    rR + Grr == GrrrR | (ky2, dy2)

    GR  + R  == GRR   | (ky3, dy3)
    GR  + rR == GrRR  | (ky3, dy3)
    GrR + R  == GrRR  | (ky3, dy3)
    GrR + rR == GrrRR | (ky3, dy3)

    Grr   == GrR   | (ka1*L, da1)
    Grrr  == GrrR  | (ka1*L, da1)
    Grrrr == GrrrR | (ka1*L, da1)

    GrR   == GRR   | (ka2*L, da2)
    GrrR  == GrRR  | (ka2*L, da2)
    GrrrR == GrrRR | (ka2*L, da2)

m = get_model()
show(m)

#### Trivalent model (N = 3 and kx > 0)

In [ ]:
# this is the trivalent model : N=3
N = 3.00

mols = ["r", "rr", "rrr", "rrrr", "rrrrr", "rrrrrr", \
        "R", "rR", "rrR", "rrrR", "rrrrR", "rrrrrR", \
        "RR", "rRR", "rrRR", "rrrRR", "rrrrRR", \
        "Gr", "Grr", "Grrr", "Grrrr", "Grrrrr", "Grrrrrr", \
        "GR", "GrR", "GrrR", "GrrrR", "GrrrrR", "GrrrrrR", \
        "GRR", "GrRR", "GrrRR", "GrrrRR", "GrrrrRR"]

In [ ]:
with reaction_rules():

    # TCM (Ternary complex model)
    r  == R  | (kl0*L, dl0)
    R  == GR | (ka0*G, da0)
    r  == Gr | (kg0*G, dg0)
    Gr == GR | (kb0*L, db0)

    # receptor dimerization
    r   + r   == rr     | (kx0, dx0)
    r   + rr  == rrr    | (kx0, dx0)
    r   + rrr == rrrr   | (kx0, dx0)
    rr  + r   == rrr    | (kx0, dx0)
    rr  + rr  == rrrr   | (kx0, dx0)
    rr  + rrr == rrrrr  | (kx0, dx0)
    rrr + r   == rrrr   | (kx0, dx0)
    rrr + rr  == rrrrr  | (kx0, dx0)
    rrr + rrr == rrrrrr | (kx0, dx0)

    R   + r   == rR     | (kx1, dx1)
    R   + rr  == rrR    | (kx1, dx1)
    R   + rrr == rrrR   | (kx1, dx1)
    rR  + r   == rrR    | (kx1, dx1)
    rR  + rr  == rrrR   | (kx1, dx1)
    rR  + rrr == rrrrR  | (kx1, dx1)
    rrR + r   == rrrR   | (kx1, dx1)
    rrR + rr  == rrrrR  | (kx1, dx1)
    rrR + rrr == rrrrrR | (kx1, dx1)

    R   + R   == RR     | (kx2, dx2)
    R   + rR  == rRR    | (kx2, dx2)
    R   + rrR == rrRR   | (kx2, dx2)
    rR  + R   == rRR    | (kx2, dx2)
    rR  + rR  == rrRR   | (kx2, dx2)
    rR  + rrR == rrrRR  | (kx2, dx2)
    rrR + R   == rrRR   | (kx2, dx2)
    rrR + rR  == rrrRR  | (kx2, dx2)
    rrR + rrR == rrrrRR | (kx2, dx2)

    rr     == rR     | (kl1*L, dl1)
    rrr    == rrR    | (kl1*L, dl1)
    rrrr   == rrrR   | (kl1*L, dl1)
    rrrrr  == rrrrR  | (kl1*L, dl1)
    rrrrrr == rrrrrR | (kl1*L, dl1)

    rR     == RR     | (kl2*L, dl2)
    rrR    == rRR    | (kl2*L, dl2)
    rrrR   == rrRR   | (kl2*L, dl2)
    rrrrR  == rrrRR  | (kl2*L, dl2)
    rrrrrR == rrrrRR | (kl2*L, dl2)

    # dimers coupled with G-proteins
    rr     == Grr     | (kg1*G, dg1)
    rrr    == Grrr    | (kg1*G, dg1)
    rrrr   == Grrrr   | (kg1*G, dg1)
    rrrrr  == Grrrrr  | (kg1*G, dg1)
    rrrrrr == Grrrrrr | (kg1*G, dg1)

    rR     == GrR     | (kb1*G, db1)
    rrR    == GrrR    | (kb1*G, db1)
    rrrR   == GrrrR   | (kb1*G, db1)
    rrrrR  == GrrrrR  | (kb1*G, db1)
    rrrrrR == GrrrrrR | (kb1*G, db1)

    RR     == GRR     | (kb2*G, db2)
    rRR    == GrRR    | (kb2*G, db2)
    rrRR   == GrrRR   | (kb2*G, db2)
    rrrRR  == GrrrRR  | (kb2*G, db2)
    rrrrRR == GrrrrRR | (kb2*G, db2)

    Gr   + r   == Grr     | (ky0, dy0)
    Gr   + rr  == Grrr    | (ky0, dy0)
    Gr   + rrr == Grrrr   | (ky0, dy0)
    Grr  + r   == Grrr    | (ky0, dy0)
    Grr  + rr  == Grrrr   | (ky0, dy0)
    Grr  + rrr == Grrrrr  | (ky0, dy0)
    Grrr + r   == Grrrr   | (ky0, dy0)
    Grrr + rr  == Grrrrr  | (ky0, dy0)
    Grrr + rrr == Grrrrrr | (ky0, dy0)

    GR   + r   == GrR     | (ky1, dy1)
    GR   + rr  == GrrR    | (ky1, dy1)
    GR   + rrr == GrrrR   | (ky1, dy1)
    GrR  + r   == GrrR    | (ky1, dy1)
    GrR  + rr  == GrrrR   | (ky1, dy1)
    GrR  + rrr == GrrrrR  | (ky1, dy1)
    GrrR + r   == GrrrR   | (ky1, dy1)
    GrrR + rr  == GrrrrR  | (ky1, dy1)
    GrrR + rrr == GrrrrrR | (ky1, dy1)

    R   + Gr   == GrR     | (ky2, dy2)
    R   + Grr  == GrrR    | (ky2, dy2)
    R   + Grrr == GrrrR   | (ky2, dy2)
    rR  + Gr   == GrrR    | (ky2, dy2)
    rR  + Grr  == GrrrR   | (ky2, dy2)
    rR  + Grrr == GrrrrR  | (ky2, dy2)
    rrR + Gr   == GrrrR   | (ky2, dy2)
    rrR + Grr  == GrrrrR  | (ky2, dy2)
    rrR + Grrr == GrrrrrR | (ky2, dy2)

    GR   + R   == GRR     | (ky3, dy3)
    GR   + rR  == GrRR    | (ky3, dy3)
    GR   + rrR == GrrRR   | (ky3, dy3)
    GrR  + R   == GrRR    | (ky3, dy3)
    GrR  + rR  == GrrRR   | (ky3, dy3)
    GrR  + rrR == GrrrRR  | (ky3, dy3)
    GrrR + R   == GrrRR   | (ky3, dy3)
    GrrR + rR  == GrrrRR  | (ky3, dy3)
    GrrR + rrR == GrrrrRR | (ky3, dy3)

    Grr     == GrR     | (ka1*L, da1)
    Grrr    == GrrR    | (ka1*L, da1)
    Grrrr   == GrrrR   | (ka1*L, da1)
    Grrrrr  == GrrrrR  | (ka1*L, da1)
    Grrrrrr == GrrrrrR | (ka1*L, da1)

    GrR     == GRR     | (ka2*L, da2)
    GrrR    == GrRR    | (ka2*L, da2)
    GrrrR   == GrrRR   | (ka2*L, da2)
    GrrrrR  == GrrrRR  | (ka2*L, da2)
    GrrrrrR == GrrrrRR | (ka2*L, da2)

m = get_model()
show(m)

#### Tetravalent model (N = 4 and kx > 0)

In [ ]:
# this is the tetravalent model : N=4
N = 4.00

mols = ["r", "rr", "rrr", "rrrr", "rrrrr", "rrrrrr", "rrrrrrr", "rrrrrrrr", \
        "R", "rR", "rrR", "rrrR", "rrrrR", "rrrrrR", "rrrrrrR", "rrrrrrrR", \
        "RR", "rRR", "rrRR", "rrrRR", "rrrrRR", "rrrrrRR", "rrrrrrRR", \
        "Gr", "Grr", "Grrr", "Grrrr", "Grrrrr", "Grrrrrr", "Grrrrrrr", "Grrrrrrrr", \
        "GR", "GrR", "GrrR", "GrrrR", "GrrrrR", "GrrrrrR", "GrrrrrrR", "GrrrrrrrR", \
        "GRR", "GrRR", "GrrRR", "GrrrRR", "GrrrrRR", "GrrrrrRR", "GrrrrrrRR"]

In [ ]:
with reaction_rules():

    # TCM (Ternary complex model)
    r  == R  | (kl0*L, dl0)
    R  == GR | (ka0*G, da0)
    r  == Gr | (kg0*G, dg0)
    Gr == GR | (kb0*L, db0)

    # receptor dimerization
    r    + r    == rr       | (kx0, dx0)
    r    + rr   == rrr      | (kx0, dx0)
    r    + rrr  == rrrr     | (kx0, dx0)
    r    + rrrr == rrrrr    | (kx0, dx0)
    rr   + r    == rrr      | (kx0, dx0)
    rr   + rr   == rrrr     | (kx0, dx0)
    rr   + rrr  == rrrrr    | (kx0, dx0)
    rr   + rrrr == rrrrrr   | (kx0, dx0)
    rrr  + r    == rrrr     | (kx0, dx0)
    rrr  + rr   == rrrrr    | (kx0, dx0)
    rrr  + rrr  == rrrrrr   | (kx0, dx0)
    rrr  + rrrr == rrrrrrr  | (kx0, dx0)
    rrrr + r    == rrrrr    | (kx0, dx0)
    rrrr + rr   == rrrrrr   | (kx0, dx0)
    rrrr + rrr  == rrrrrrr  | (kx0, dx0)
    rrrr + rrrr == rrrrrrrr | (kx0, dx0)

    R    + r    == rR       | (kx1, dx1)
    R    + rr   == rrR      | (kx1, dx1)
    R    + rrr  == rrrR     | (kx1, dx1)
    R    + rrrr == rrrrR    | (kx1, dx1)
    rR   + r    == rrR      | (kx1, dx1)
    rR   + rr   == rrrR     | (kx1, dx1)
    rR   + rrr  == rrrrR    | (kx1, dx1)
    rR   + rrrr == rrrrrR   | (kx1, dx1)
    rrR  + r    == rrrR     | (kx1, dx1)
    rrR  + rr   == rrrrR    | (kx1, dx1)
    rrR  + rrr  == rrrrrR   | (kx1, dx1)
    rrR  + rrrr == rrrrrrR  | (kx1, dx1)
    rrrR + r    == rrrrR    | (kx1, dx1)
    rrrR + rr   == rrrrrR   | (kx1, dx1)
    rrrR + rrr  == rrrrrrR  | (kx1, dx1)
    rrrR + rrrr == rrrrrrrR | (kx1, dx1)

    R    + R    == RR       | (kx2, dx2)
    R    + rR   == rRR      | (kx2, dx2)
    R    + rrR  == rrRR     | (kx2, dx2)
    R    + rrrR == rrrRR    | (kx2, dx2)
    rR   + R    == rRR      | (kx2, dx2)
    rR   + rR   == rrRR     | (kx2, dx2)
    rR   + rrR  == rrrRR    | (kx2, dx2)
    rR   + rrrR == rrrrRR   | (kx2, dx2)
    rrR  + R    == rrRR     | (kx2, dx2)
    rrR  + rR   == rrrRR    | (kx2, dx2)
    rrR  + rrR  == rrrrRR   | (kx2, dx2)
    rrR  + rrrR == rrrrrRR  | (kx2, dx2)
    rrrR + R    == rrrRR    | (kx2, dx2)
    rrrR + rR   == rrrrRR   | (kx2, dx2)
    rrrR + rrR  == rrrrrRR  | (kx2, dx2)
    rrrR + rrrR == rrrrrrRR | (kx2, dx2)

    rr       == rR       | (kl1*L, dl1)
    rrr      == rrR      | (kl1*L, dl1)
    rrrr     == rrrR     | (kl1*L, dl1)
    rrrrr    == rrrrR    | (kl1*L, dl1)
    rrrrrr   == rrrrrR   | (kl1*L, dl1)
    rrrrrrr  == rrrrrrR  | (kl1*L, dl1)
    rrrrrrrr == rrrrrrrR | (kl1*L, dl1)

    rR       == RR       | (kl2*L, dl2)
    rrR      == rRR      | (kl2*L, dl2)
    rrrR     == rrRR     | (kl2*L, dl2)
    rrrrR    == rrrRR    | (kl2*L, dl2)
    rrrrrR   == rrrrRR   | (kl2*L, dl2)
    rrrrrrR  == rrrrrRR  | (kl2*L, dl2)
    rrrrrrrR == rrrrrrRR | (kl2*L, dl2)

    # dimers coupled with G-proteins
    rr       == Grr       | (kg1*G, dg1)
    rrr      == Grrr      | (kg1*G, dg1)
    rrrr     == Grrrr     | (kg1*G, dg1)
    rrrrr    == Grrrrr    | (kg1*G, dg1)
    rrrrrr   == Grrrrrr   | (kg1*G, dg1)
    rrrrrrr  == Grrrrrrr  | (kg1*G, dg1)
    rrrrrrrr == Grrrrrrrr | (kg1*G, dg1)

    rR       == GrR       | (kb1*G, db1)
    rrR      == GrrR      | (kb1*G, db1)
    rrrR     == GrrrR     | (kb1*G, db1)
    rrrrR    == GrrrrR    | (kb1*G, db1)
    rrrrrR   == GrrrrrR   | (kb1*G, db1)
    rrrrrrR  == GrrrrrrR  | (kb1*G, db1)
    rrrrrrrR == GrrrrrrrR | (kb1*G, db1)

    RR       == GRR       | (kb2*G, db2)
    rRR      == GrRR      | (kb2*G, db2)
    rrRR     == GrrRR     | (kb2*G, db2)
    rrrRR    == GrrrRR    | (kb2*G, db2)
    rrrrRR   == GrrrrRR   | (kb2*G, db2)
    rrrrrRR  == GrrrrrRR  | (kb2*G, db2)
    rrrrrrRR == GrrrrrrRR | (kb2*G, db2)

    Gr    + r    == Grr       | (ky0, dy0)
    Gr    + rr   == Grrr      | (ky0, dy0)
    Gr    + rrr  == Grrrr     | (ky0, dy0)
    Gr    + rrrr == Grrrrr    | (ky0, dy0)
    Grr   + r    == Grrr      | (ky0, dy0)
    Grr   + rr   == Grrrr     | (ky0, dy0)
    Grr   + rrr  == Grrrrr    | (ky0, dy0)
    Grr   + rrrr == Grrrrrr   | (ky0, dy0)
    Grrr  + r    == Grrrr     | (ky0, dy0)
    Grrr  + rr   == Grrrrr    | (ky0, dy0)
    Grrr  + rrr  == Grrrrrr   | (ky0, dy0)
    Grrr  + rrrr == Grrrrrrr  | (ky0, dy0)
    Grrrr + r    == Grrrrr    | (ky0, dy0)
    Grrrr + rr   == Grrrrrr   | (ky0, dy0)
    Grrrr + rrr  == Grrrrrrr  | (ky0, dy0)
    Grrrr + rrrr == Grrrrrrrr | (ky0, dy0)

    GR    + r    == GrR       | (ky1, dy1)
    GR    + rr   == GrrR      | (ky1, dy1)
    GR    + rrr  == GrrrR     | (ky1, dy1)
    GR    + rrrr == GrrrrR    | (ky1, dy1)
    GrR   + r    == GrrR      | (ky1, dy1)
    GrR   + rr   == GrrrR     | (ky1, dy1)
    GrR   + rrr  == GrrrrR    | (ky1, dy1)
    GrR   + rrrr == GrrrrrR   | (ky1, dy1)
    GrrR  + r    == GrrrR     | (ky1, dy1)
    GrrR  + rr   == GrrrrR    | (ky1, dy1)
    GrrR  + rrr  == GrrrrrR   | (ky1, dy1)
    GrrR  + rrrr == GrrrrrrR  | (ky1, dy1)
    GrrrR + r    == GrrrrR    | (ky1, dy1)
    GrrrR + rr   == GrrrrrR   | (ky1, dy1)
    GrrrR + rrr  == GrrrrrrR  | (ky1, dy1)
    GrrrR + rrrr == GrrrrrrrR | (ky1, dy1)


    R    + Gr    == GrR       | (ky2, dy2)
    R    + Grr   == GrrR      | (ky2, dy2)
    R    + Grrr  == GrrrR     | (ky2, dy2)
    R    + Grrrr == GrrrrR    | (ky2, dy2)
    rR   + Gr    == GrrR      | (ky2, dy2)
    rR   + Grr   == GrrrR     | (ky2, dy2)
    rR   + Grrr  == GrrrrR    | (ky2, dy2)
    rR   + Grrrr == GrrrrrR   | (ky2, dy2)
    rrR  + Gr    == GrrrR     | (ky2, dy2)
    rrR  + Grr   == GrrrrR    | (ky2, dy2)
    rrR  + Grrr  == GrrrrrR   | (ky2, dy2)
    rrR  + Grrrr == GrrrrrrR  | (ky2, dy2)
    rrrR + Gr    == GrrrrR    | (ky2, dy2)
    rrrR + Grr   == GrrrrrR   | (ky2, dy2)
    rrrR + Grrr  == GrrrrrrR  | (ky2, dy2)
    rrrR + Grrrr == GrrrrrrrR | (ky2, dy2)

    GR    + R    == GRR       | (ky3, dy3)
    GR    + rR   == GrRR      | (ky3, dy3)
    GR    + rrR  == GrrRR     | (ky3, dy3)
    GR    + rrrR == GrrrRR    | (ky3, dy3)
    GrR   + R    == GrRR      | (ky3, dy3)
    GrR   + rR   == GrrRR     | (ky3, dy3)
    GrR   + rrR  == GrrrRR    | (ky3, dy3)
    GrR   + rrrR == GrrrrRR   | (ky3, dy3)
    GrrR  + R    == GrrRR     | (ky3, dy3)
    GrrR  + rR   == GrrrRR    | (ky3, dy3)
    GrrR  + rrR  == GrrrrRR   | (ky3, dy3)
    GrrR  + rrrR == GrrrrrRR  | (ky3, dy3)
    GrrrR + R    == GrrrRR    | (ky3, dy3)
    GrrrR + rR   == GrrrrRR   | (ky3, dy3)
    GrrrR + rrR  == GrrrrrRR  | (ky3, dy3)
    GrrrR + rrrR == GrrrrrrRR | (ky3, dy3)

    Grr       == GrR       | (ka1*L, da1)
    Grrr      == GrrR      | (ka1*L, da1)
    Grrrr     == GrrrR     | (ka1*L, da1)
    Grrrrr    == GrrrrR    | (ka1*L, da1)
    Grrrrrr   == GrrrrrR   | (ka1*L, da1)
    Grrrrrrr  == GrrrrrrR  | (ka1*L, da1)
    Grrrrrrrr == GrrrrrrrR | (ka1*L, da1)

    GrR       == GRR       | (ka2*L, da2)
    GrrR      == GrRR      | (ka2*L, da2)
    GrrrR     == GrrRR     | (ka2*L, da2)
    GrrrrR    == GrrrRR    | (ka2*L, da2)
    GrrrrrR   == GrrrrRR   | (ka2*L, da2)
    GrrrrrrR  == GrrrrrRR  | (ka2*L, da2)
    GrrrrrrrR == GrrrrrrRR | (ka2*L, da2)

m = get_model()
show(m)

#### Pentavalent model (N = 5 and kx > 0)

In [ ]:
# this is the pentavalent model : N=5
N = 5.00

mols =  ["r", "rr", "rrr", "rrrr", "rrrrr", "rrrrrr", "rrrrrrr", "rrrrrrrr", "rrrrrrrr", "rrrrrrrrrr", \
         "R", "rR", "rrR", "rrrR", "rrrrR", "rrrrrR", "rrrrrrR", "rrrrrrrR", "rrrrrrrR", "rrrrrrrrrR", \
         "RR", "rRR", "rrRR", "rrrRR", "rrrrRR", "rrrrrRR", "rrrrrrRR", "rrrrrrRR", "rrrrrrrrRR", \
         "Gr", "Grr", "Grrr", "Grrrr", "Grrrrr", "Grrrrrr", "Grrrrrrr", "Grrrrrrrr", "Grrrrrrrr", "Grrrrrrrrrr", \
         "GR", "GrR", "GrrR", "GrrrR", "GrrrrR", "GrrrrrR", "GrrrrrrR", "GrrrrrrrR", "GrrrrrrrR", "GrrrrrrrrrR", \
         "GRR", "GrRR", "GrrRR", "GrrrRR", "GrrrrRR", "GrrrrrRR", "GrrrrrrRR", "GrrrrrrRR", "GrrrrrrrrRR"]

In [ ]:
with reaction_rules():

    # TCM (Ternary complex model)
    r  == R  | (kl0*L, dl0)
    R  == GR | (ka0*G, da0)
    r  == Gr | (kg0*G, dg0)
    Gr == GR | (kb0*L, db0)

    # receptor dimerization
    r     + r     == rr         | (kx0, dx0)
    r     + rr    == rrr        | (kx0, dx0)
    r     + rrr   == rrrr       | (kx0, dx0)
    r     + rrrr  == rrrrr      | (kx0, dx0)
    r     + rrrrr == rrrrrr     | (kx0, dx0)
    rr    + r     == rrr        | (kx0, dx0)
    rr    + rr    == rrrr       | (kx0, dx0)
    rr    + rrr   == rrrrr      | (kx0, dx0)
    rr    + rrrr  == rrrrrr     | (kx0, dx0)
    rr    + rrrrr == rrrrrrr    | (kx0, dx0)
    rrr   + r     == rrrr       | (kx0, dx0)
    rrr   + rr    == rrrrr      | (kx0, dx0)
    rrr   + rrr   == rrrrrr     | (kx0, dx0)
    rrr   + rrrr  == rrrrrrr    | (kx0, dx0)
    rrr   + rrrrr == rrrrrrrr   | (kx0, dx0)
    rrrr  + r     == rrrrr      | (kx0, dx0)
    rrrr  + rr    == rrrrrr     | (kx0, dx0)
    rrrr  + rrr   == rrrrrrr    | (kx0, dx0)
    rrrr  + rrrr  == rrrrrrrr   | (kx0, dx0)
    rrrr  + rrrrr == rrrrrrrrr  | (kx0, dx0)
    rrrrr + r     == rrrrrr     | (kx0, dx0)
    rrrrr + rr    == rrrrrrr    | (kx0, dx0)
    rrrrr + rrr   == rrrrrrrr   | (kx0, dx0)
    rrrrr + rrrr  == rrrrrrrrr  | (kx0, dx0)
    rrrrr + rrrrr == rrrrrrrrrr | (kx0, dx0)

    R     + r     == rR         | (kx1, dx1)
    R     + rr    == rrR        | (kx1, dx1)
    R     + rrr   == rrrR       | (kx1, dx1)
    R     + rrrr  == rrrrR      | (kx1, dx1)
    R     + rrrrr == rrrrrR     | (kx1, dx1)
    rR    + r     == rrR        | (kx1, dx1)
    rR    + rr    == rrrR       | (kx1, dx1)
    rR    + rrr   == rrrrR      | (kx1, dx1)
    rR    + rrrr  == rrrrrR     | (kx1, dx1)
    rR    + rrrrr == rrrrrrR    | (kx1, dx1)
    rrR   + r     == rrrR       | (kx1, dx1)
    rrR   + rr    == rrrrR      | (kx1, dx1)
    rrR   + rrr   == rrrrrR     | (kx1, dx1)
    rrR   + rrrr  == rrrrrrR    | (kx1, dx1)
    rrR   + rrrrr == rrrrrrrR   | (kx1, dx1)
    rrrR  + r     == rrrrR      | (kx1, dx1)
    rrrR  + rr    == rrrrrR     | (kx1, dx1)
    rrrR  + rrr   == rrrrrrR    | (kx1, dx1)
    rrrR  + rrrr  == rrrrrrrR   | (kx1, dx1)
    rrrR  + rrrrr == rrrrrrrrR  | (kx1, dx1)
    rrrrR + r     == rrrrrR     | (kx1, dx1)
    rrrrR + rr    == rrrrrrR    | (kx1, dx1)
    rrrrR + rrr   == rrrrrrrR   | (kx1, dx1)
    rrrrR + rrrr  == rrrrrrrrR  | (kx1, dx1)
    rrrrR + rrrrr == rrrrrrrrrR | (kx1, dx1)

    R     + R     == RR         | (kx2, dx2)
    R     + rR    == rRR        | (kx2, dx2)
    R     + rrR   == rrRR       | (kx2, dx2)
    R     + rrrR  == rrrRR      | (kx2, dx2)
    R     + rrrrR == rrrrRR     | (kx2, dx2)
    rR    + R     == rRR        | (kx2, dx2)
    rR    + rR    == rrRR       | (kx2, dx2)
    rR    + rrR   == rrrRR      | (kx2, dx2)
    rR    + rrrR  == rrrrRR     | (kx2, dx2)
    rR    + rrrrR == rrrrrRR    | (kx2, dx2)
    rrR   + R     == rrRR       | (kx2, dx2)
    rrR   + rR    == rrrRR      | (kx2, dx2)
    rrR   + rrR   == rrrrRR     | (kx2, dx2)
    rrR   + rrrR  == rrrrrRR    | (kx2, dx2)
    rrR   + rrrrR == rrrrrrRR   | (kx2, dx2)
    rrrR  + R     == rrrRR      | (kx2, dx2)
    rrrR  + rR    == rrrrRR     | (kx2, dx2)
    rrrR  + rrR   == rrrrrRR    | (kx2, dx2)
    rrrR  + rrrR  == rrrrrrRR   | (kx2, dx2)
    rrrR  + rrrrR == rrrrrrrRR  | (kx2, dx2)
    rrrrR + R     == rrrrRR     | (kx2, dx2)
    rrrrR + rR    == rrrrrRR    | (kx2, dx2)
    rrrrR + rrR   == rrrrrrRR   | (kx2, dx2)
    rrrrR + rrrR  == rrrrrrrRR  | (kx2, dx2)
    rrrrR + rrrrR == rrrrrrrrRR | (kx2, dx2)

    rr         == rR         | (kl1*L, dl1)
    rrr        == rrR        | (kl1*L, dl1)
    rrrr       == rrrR       | (kl1*L, dl1)
    rrrrr      == rrrrR      | (kl1*L, dl1)
    rrrrrr     == rrrrrR     | (kl1*L, dl1)
    rrrrrrr    == rrrrrrR    | (kl1*L, dl1)
    rrrrrrrr   == rrrrrrrR   | (kl1*L, dl1)
    rrrrrrrrr  == rrrrrrrrR  | (kl1*L, dl1)
    rrrrrrrrrr == rrrrrrrrrR | (kl1*L, dl1)

    rR         == RR         | (kl2*L, dl2)
    rrR        == rRR        | (kl2*L, dl2)
    rrrR       == rrRR       | (kl2*L, dl2)
    rrrrR      == rrrRR      | (kl2*L, dl2)
    rrrrrR     == rrrrRR     | (kl2*L, dl2)
    rrrrrrR    == rrrrrRR    | (kl2*L, dl2)
    rrrrrrrR   == rrrrrrRR   | (kl2*L, dl2)
    rrrrrrrrR  == rrrrrrrRR  | (kl2*L, dl2)
    rrrrrrrrrR == rrrrrrrrRR | (kl2*L, dl2)

    # dimers coupled with G-proteins
    rr         == Grr         | (kg1*G, dg1)
    rrr        == Grrr        | (kg1*G, dg1)
    rrrr       == Grrrr       | (kg1*G, dg1)
    rrrrr      == Grrrrr      | (kg1*G, dg1)
    rrrrrr     == Grrrrrr     | (kg1*G, dg1)
    rrrrrrr    == Grrrrrrr    | (kg1*G, dg1)
    rrrrrrrr   == Grrrrrrrr   | (kg1*G, dg1)
    rrrrrrrrr  == Grrrrrrrrr  | (kg1*G, dg1)
    rrrrrrrrrr == Grrrrrrrrrr | (kg1*G, dg1)

    rR         == GrR         | (kb1*G, db1)
    rrR        == GrrR        | (kb1*G, db1)
    rrrR       == GrrrR       | (kb1*G, db1)
    rrrrR      == GrrrrR      | (kb1*G, db1)
    rrrrrR     == GrrrrrR     | (kb1*G, db1)
    rrrrrrR    == GrrrrrrR    | (kb1*G, db1)
    rrrrrrrR   == GrrrrrrrR   | (kb1*G, db1)
    rrrrrrrrR  == GrrrrrrrrR  | (kb1*G, db1)
    rrrrrrrrrR == GrrrrrrrrrR | (kb1*G, db1)

    RR         == GRR         | (kb2*G, db2)
    rRR        == GrRR        | (kb2*G, db2)
    rrRR       == GrrRR       | (kb2*G, db2)
    rrrRR      == GrrrRR      | (kb2*G, db2)
    rrrrRR     == GrrrrRR     | (kb2*G, db2)
    rrrrrRR    == GrrrrrRR    | (kb2*G, db2)
    rrrrrrRR   == GrrrrrrRR   | (kb2*G, db2)
    rrrrrrrRR  == GrrrrrrrRR  | (kb2*G, db2)
    rrrrrrrrRR == GrrrrrrrrRR | (kb2*G, db2)

    Gr     + r     == Grr         | (ky0, dy0)
    Gr     + rr    == Grrr        | (ky0, dy0)
    Gr     + rrr   == Grrrr       | (ky0, dy0)
    Gr     + rrrr  == Grrrrr      | (ky0, dy0)
    Gr     + rrrrr == Grrrrrr     | (ky0, dy0)
    Grr    + r     == Grrr        | (ky0, dy0)
    Grr    + rr    == Grrrr       | (ky0, dy0)
    Grr    + rrr   == Grrrrr      | (ky0, dy0)
    Grr    + rrrr  == Grrrrrr     | (ky0, dy0)
    Grr    + rrrrr == Grrrrrrr    | (ky0, dy0)
    Grrr   + r     == Grrrr       | (ky0, dy0)
    Grrr   + rr    == Grrrrr      | (ky0, dy0)
    Grrr   + rrr   == Grrrrrr     | (ky0, dy0)
    Grrr   + rrrr  == Grrrrrrr    | (ky0, dy0)
    Grrr   + rrrrr == Grrrrrrrr   | (ky0, dy0)
    Grrrr  + r     == Grrrrr      | (ky0, dy0)
    Grrrr  + rr    == Grrrrrr     | (ky0, dy0)
    Grrrr  + rrr   == Grrrrrrr    | (ky0, dy0)
    Grrrr  + rrrr  == Grrrrrrrr   | (ky0, dy0)
    Grrrr  + rrrrr == Grrrrrrrrr  | (ky0, dy0)
    Grrrrr + r     == Grrrrrr     | (ky0, dy0)
    Grrrrr + rr    == Grrrrrrr    | (ky0, dy0)
    Grrrrr + rrr   == Grrrrrrrr   | (ky0, dy0)
    Grrrrr + rrrr  == Grrrrrrrrr  | (ky0, dy0)
    Grrrrr + rrrrr == Grrrrrrrrrr | (ky0, dy0)

    GR     + r     == GrR         | (ky1, dy1)
    GR     + rr    == GrrR        | (ky1, dy1)
    GR     + rrr   == GrrrR       | (ky1, dy1)
    GR     + rrrr  == GrrrrR      | (ky1, dy1)
    GR     + rrrrr == GrrrrrR     | (ky1, dy1)
    GrR    + r     == GrrR        | (ky1, dy1)
    GrR    + rr    == GrrrR       | (ky1, dy1)
    GrR    + rrr   == GrrrrR      | (ky1, dy1)
    GrR    + rrrr  == GrrrrrR     | (ky1, dy1)
    GrR    + rrrrr == GrrrrrrR    | (ky1, dy1)
    GrrR   + r     == GrrrR       | (ky1, dy1)
    GrrR   + rr    == GrrrrR      | (ky1, dy1)
    GrrR   + rrr   == GrrrrrR     | (ky1, dy1)
    GrrR   + rrrr  == GrrrrrrR    | (ky1, dy1)
    GrrR   + rrrrr == GrrrrrrrR   | (ky1, dy1)
    GrrrR  + r     == GrrrrR      | (ky1, dy1)
    GrrrR  + rr    == GrrrrrR     | (ky1, dy1)
    GrrrR  + rrr   == GrrrrrrR    | (ky1, dy1)
    GrrrR  + rrrr  == GrrrrrrrR   | (ky1, dy1)
    GrrrR  + rrrrr == GrrrrrrrrR  | (ky1, dy1)
    GrrrrR + r     == GrrrrrR     | (ky1, dy1)
    GrrrrR + rr    == GrrrrrrR    | (ky1, dy1)
    GrrrrR + rrr   == GrrrrrrrR   | (ky1, dy1)
    GrrrrR + rrrr  == GrrrrrrrrR  | (ky1, dy1)
    GrrrrR + rrrrr == GrrrrrrrrrR | (ky1, dy1)


    R     + Gr     == GrR         | (ky2, dy2)
    R     + Grr    == GrrR        | (ky2, dy2)
    R     + Grrr   == GrrrR       | (ky2, dy2)
    R     + Grrrr  == GrrrrR      | (ky2, dy2)
    R     + Grrrrr == GrrrrrR     | (ky2, dy2)
    rR    + Gr     == GrrR        | (ky2, dy2)
    rR    + Grr    == GrrrR       | (ky2, dy2)
    rR    + Grrr   == GrrrrR      | (ky2, dy2)
    rR    + Grrrr  == GrrrrrR     | (ky2, dy2)
    rR    + Grrrrr == GrrrrrrR    | (ky2, dy2)
    rrR   + Gr     == GrrrR       | (ky2, dy2)
    rrR   + Grr    == GrrrrR      | (ky2, dy2)
    rrR   + Grrr   == GrrrrrR     | (ky2, dy2)
    rrR   + Grrrr  == GrrrrrrR    | (ky2, dy2)
    rrR   + Grrrrr == GrrrrrrrR   | (ky2, dy2)
    rrrR  + Gr     == GrrrrR      | (ky2, dy2)
    rrrR  + Grr    == GrrrrrR     | (ky2, dy2)
    rrrR  + Grrr   == GrrrrrrR    | (ky2, dy2)
    rrrR  + Grrrr  == GrrrrrrrR   | (ky2, dy2)
    rrrR  + Grrrrr == GrrrrrrrrR  | (ky2, dy2)
    rrrrR + Gr     == GrrrrrR     | (ky2, dy2)
    rrrrR + Grr    == GrrrrrrR    | (ky2, dy2)
    rrrrR + Grrr   == GrrrrrrrR   | (ky2, dy2)
    rrrrR + Grrrr  == GrrrrrrrrR  | (ky2, dy2)
    rrrrR + Grrrrr == GrrrrrrrrrR | (ky2, dy2)

    GR     + R     == GRR         | (ky3, dy3)
    GR     + rR    == GrRR        | (ky3, dy3)
    GR     + rrR   == GrrRR       | (ky3, dy3)
    GR     + rrrR  == GrrrRR      | (ky3, dy3)
    GR     + rrrrR == GrrrrRR     | (ky3, dy3)
    GrR    + R     == GrRR        | (ky3, dy3)
    GrR    + rR    == GrrRR       | (ky3, dy3)
    GrR    + rrR   == GrrrRR      | (ky3, dy3)
    GrR    + rrrR  == GrrrrRR     | (ky3, dy3)
    GrR    + rrrrR == GrrrrrRR    | (ky3, dy3)
    GrrR   + R     == GrrRR       | (ky3, dy3)
    GrrR   + rR    == GrrrRR      | (ky3, dy3)
    GrrR   + rrR   == GrrrrRR     | (ky3, dy3)
    GrrR   + rrrR  == GrrrrrRR    | (ky3, dy3)
    GrrR   + rrrrR == GrrrrrrRR   | (ky3, dy3)
    GrrrR  + R     == GrrrRR      | (ky3, dy3)
    GrrrR  + rR    == GrrrrRR     | (ky3, dy3)
    GrrrR  + rrR   == GrrrrrRR    | (ky3, dy3)
    GrrrR  + rrrR  == GrrrrrrRR   | (ky3, dy3)
    GrrrR  + rrrrR == GrrrrrrrRR  | (ky3, dy3)
    GrrrrR + R     == GrrrrRR     | (ky3, dy3)
    GrrrrR + rR    == GrrrrrRR    | (ky3, dy3)
    GrrrrR + rrR   == GrrrrrrRR   | (ky3, dy3)
    GrrrrR + rrrR  == GrrrrrrrRR  | (ky3, dy3)
    GrrrrR + rrrrR == GrrrrrrrrRR | (ky3, dy3)

    Grr         == GrR         | (ka1*L, da1)
    Grrr        == GrrR        | (ka1*L, da1)
    Grrrr       == GrrrR       | (ka1*L, da1)
    Grrrrr      == GrrrrR      | (ka1*L, da1)
    Grrrrrr     == GrrrrrR     | (ka1*L, da1)
    Grrrrrrr    == GrrrrrrR    | (ka1*L, da1)
    Grrrrrrrr   == GrrrrrrrR   | (ka1*L, da1)
    Grrrrrrrrr  == GrrrrrrrrR  | (ka1*L, da1)
    Grrrrrrrrrr == GrrrrrrrrrR | (ka1*L, da1)

    GrR         == GRR         | (ka2*L, da2)
    GrrR        == GrRR        | (ka2*L, da2)
    GrrrR       == GrrRR       | (ka2*L, da2)
    GrrrrR      == GrrrRR      | (ka2*L, da2)
    GrrrrrR     == GrrrrRR     | (ka2*L, da2)
    GrrrrrrR    == GrrrrrRR    | (ka2*L, da2)
    GrrrrrrrR   == GrrrrrrRR   | (ka2*L, da2)
    GrrrrrrrrR  == GrrrrrrrRR  | (ka2*L, da2)
    GrrrrrrrrrR == GrrrrrrrrRR | (ka2*L, da2)

m = get_model()
show(m)